In [2]:
from pyplasm import *

Evaluating fenvs.py..
...fenvs.py imported in 0.0136186529144 seconds


In [ ]:
vertici=[[0,0,0],[15,0,0],[15,10,0],[0,10,0],[5,5,0],[10,5,0],[5,5,1.5],[10,5,1.5]]
celle=[1,2,3,4,5,6,7,8]
tetto1=MKPOL([vertici,[celle],None])
VIEW(tetto1)